# Stock Market Alert App

### Introduction

As someone who has never really been interested in the stock market, I nevertheless was curious about this project which made me create a system that not only tracks stock price changes but also keeps me updated with the latest news related to those changes. The goal was to build a practical tool that could help me stay informed about significant fluctuations in stock prices and their potential causes.

### The Challenges I Faced

- Handling API Data: One of the first challenges was to correctly fetch and process stock data from the Alpha Vantage API. It required careful parsing of the JSON responses to extract relevant information such as closing prices for consecutive days.
- Calculating Price Changes: Calculating the percentage change in stock prices posed its own set of difficulties. Ensuring accurate calculations and handling edge cases, such as very small or very large differences, was crucial.
- Fetching and Processing News: Integrating the News API to fetch articles related to significant stock price changes required careful setup and parameter tuning. It was challenging to filter and format the news to be informative and concise.
- Sending SMS Alerts: Finally, integrating Twilio to send SMS alerts was a significant milestone. Configuring the Twilio API and handling message formatting were key aspects of this step. Ensuring that each message was clear and informative while keeping within the SMS character limits was a crucial part of the process.

### Requirements
- Importing Libraries: Setting up and securely managing sensitive information using environment variables.
- Fetching Stock Data: Retrieving daily stock prices for a specific company.
- Calculating Changes: Determining the percentage change in stock prices and assessing its significance.
- Fetching News: Obtaining related news articles if the price change exceeds a certain threshold.
- Sending Alerts: Formatting the articles and sending them via SMS using Twilio.

### Important: Implement Your Own Information

To run this notebook successfully, you will need to replace the placeholders with your own details:
- API Keys: You need to obtain and set your own API keys from Alpha Vantage and News API. Replace STOCK_API_KEY and NEWS_API_KEY with your personal keys.
- Twilio Credentials: Sign up for a Twilio account and get your own Twilio SID and Auth Token. Replace TWILIO_SID and TWILIO_AUTH_TOKEN with these values.
- Phone Numbers: Update VIRTUAL_TWILIO_NUMBER with your Twilio phone number and VERIFIED_NUMBER with the phone number where you want to receive alerts.

You can securely store these values as environment variables or directly replace them in the code. Make sure to keep your sensitive information private to avoid any security risks.

## Import Libraries

In [ ]:
from dotenv import load_dotenv
from twilio.rest import Client

In [ ]:
import os
import requests

- `os` for accessing environment variables securely
- `requests` for making HTTP requests to APIs
- `twilio.rest.Client` for interacting with the Twilio API to send SMS messages
- 'load_dotenv' for loading environment variables from a .env file

## Create A .env File

In [ ]:
STOCK_API_KEY=your_alpha_vantage_api_key
NEWS_API_KEY=your_news_api_key
TWILIO_SID=your_twilio_sid
TWILIO_AUTH_TOKEN=your_twilio_auth_token
VIRTUAL_TWILIO_NUMBER=your_twilio_phone_number
VERIFIED_NUMBER=your_verified_phone_number

## Set Environment Variables

In [ ]:
load_dotenv()

STOCK_API_KEY = os.getenv("STOCK_API_KEY")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")
TWILIO_SID = os.getenv("TWILIO_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")
VIRTUAL_TWILIO_NUMBER = os.getenv("VIRTUAL_TWILIO_NUMBER")
VERIFIED_NUMBER = os.getenv("VERIFIED_NUMBER")

- Sets environment variables for this notebook (not recommended for production due to security risks)

## Access Environment Variables

In [ ]:
VIRTUAL_TWILIO_NUMBER = os.getenv("VIRTUAL_TWILIO_NUMBER")
VERIFIED_NUMBER = os.getenv("VERIFIED_NUMBER")
STOCK_API_KEY = os.getenv("STOCK_API_KEY")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")
TWILIO_SID = os.getenv("TWILIO_SID")
TWILIO_AUTH_TOKEN = os.getenv("TWILIO_AUTH_TOKEN")

- Environment variables are used to keep sensitive information like API keys and phone numbers secure
- Replace the placeholder values with your own environment variable names or directly set the values

## Set Up Stock and Company Information

In [ ]:
STOCK_NAME = "TSLA"
COMPANY_NAME = "Tesla Inc"

STOCK_ENDPOINT = "https://www.alphavantage.co/query"
NEWS_ENDPOINT = "https://newsapi.org/v2/everything"

- `STOCK_NAME` is the ticker symbol for the stock you want to monitor (Tesla Inc in this case)
- `COMPANY_NAME` is the full name of the company for which you are fetching news
- `STOCK_ENDPOINT` and `NEWS_ENDPOINT` are the URLs for the Alpha Vantage and News API endpoints respectively

## Get Yesterday's Closing Stock Price

In [ ]:
stock_params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": STOCK_NAME,
    "apikey": STOCK_API_KEY,
}

response = requests.get(STOCK_ENDPOINT, params=stock_params)
data = response.json()["Time Series (Daily)"]

data_list = [value for (key, value) in data.items()]
yesterday_data = data_list[0]
yesterday_closing_price = yesterday_data["4. close"]

- Sends a request to the Alpha Vantage API to get the daily time series data for the specified stock
- Extracts the closing price for yesterday from the API response
- `response.json()` parses the API response as JSON
- `"Time Series (Daily)"` is the key in the response JSON containing the daily stock data
- Converts this data into a list to easily access prices for different days

## Get the Day Before Yesterday's Closing Stock Price

In [ ]:
day_before_yesterday_data = data_list[1]
day_before_yesterday_closing_price = day_before_yesterday_data["4. close"]

- Fetches the closing price for the day before yesterday from the API response
- Accesses the data for the day before yesterday by indexing the `data_list`
- `"4. close"` contains the closing price for each day

## Calculate the Difference and Percentage Change

In [ ]:
difference = float(yesterday_closing_price) - float(day_before_yesterday_closing_price)
up_down = "🔺" if difference > 0 else "🔻"

diff_percent = round((difference / float(yesterday_closing_price)) * 100)

- Calculates the difference between yesterday's and the day before yesterday's closing prices
- Computes the percentage change to assess the significance of the price movement
- `difference` is computed as the difference between the two closing prices
- `up_down` determines the symbol to indicate whether the price went up or down
- `diff_percent` calculates the percentage change and rounds it for better readability

## Fetch News Articles If the Percentage Difference is Significant
## Format the Articles and Send Them via Twilio

In [ ]:
if abs(diff_percent) > 5:
    news_params = {
        "apiKey": NEWS_API_KEY,
        "qInTitle": COMPANY_NAME,
    }

    news_response = requests.get(NEWS_ENDPOINT, params=news_params)
    articles = news_response.json()["articles"]

    three_articles = articles[:3]
    formatted_articles = [
    f"{STOCK_NAME}: {up_down}{diff_percent}%\nHeadline: {article['title']}. \nBrief: {article['description']}"
    for article in three_articles
    ]

    client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)

    for article in formatted_articles:
        message = client.messages.create(
            body=article,
            from_=VIRTUAL_TWILIO_NUMBER,
            to=VERIFIED_NUMBER
    )

- If the percentage change in stock price is greater than 5%, related news articles are fetched using the News API
- `news_params` includes the API key and a query to search for articles related to the company name
- The `news_response.json()` contains the fetched articles
- Then the list is sliced to get the top 3 articles for brevity
- Formats the top 3 articles into SMS messages and send them using the Twilio API
- `formatted_articles` is a list of strings where each string contains a formatted message with stock information and news
- `Client` is used to create a Twilio client instance
- `client.messages.create()` sends each message via SMS to the specified phone number

## Conclusion

So, I’ve just wrapped up this cool project where I built a stock price alert system, and it’s been quite a ride! Here’s the scoop on what made it both exciting and challenging:

- **Data Juggling**: I had to pull data from two different APIs – one for stock prices and one for news. Getting the data and making sure it’s accurate was like piecing together a puzzle. It was tricky to handle the JSON responses and extract the right information.

- **Crunching Numbers**: Figuring out the percentage changes in stock prices and deciding if they were significant enough to send an alert was a bit of a brain teaser. I had to ensure the calculations were spot on, which took some fine-tuning.

- **News Sorting**: Finding relevant news articles and making sure they were formatted nicely for the alerts was another challenge. I wanted to make sure the news was useful and easy to understand, which meant a lot of attention to detail.

- **Texting with Twilio**: Integrating Twilio to send SMS alerts was pretty cool but also had its quirks. I had to make sure the messages were formatted correctly and within the character limits, which took some trial and error.

Overall, it’s been a really deep learning experience! I’ve learned a lot about working with APIs, automating tasks, and making sure everything works together smoothly. I’m pretty proud of how it turned out!